In [1]:
%run base.ipynb

In [2]:
dn

SX([dx, dy, dz, dphi, dthet, dpsi])

In [3]:
J_ = Kinematics.J
J_inv = Kinematics.J_INV

In [5]:
xd = J_@x_nb
ode_xdd = uv_body.body_forward_dynamics()

rhs = vertcat(xd, ode_xdd) #the complete ODE vector
f_rhs = Function('Odefunc', [m, W, B, r_g, r_b, I_o,
                           decoupled_added_m, coupled_added_m,
                           linear_dc, quadratic_dc,
                           x_nb, v_c, eul, tau_b], [rhs])

## RETURNS vertcat(p_n ,uq , x_nb)
ode = f_rhs(blue.m, blue.W, blue.B, blue.rg, blue.rb, blue.Io, blue.added_m,
                       blue.coupl_added_m, blue.linear_dc, blue.quadratic_dc, x_nb, v_c, eul,  tau_b)

dt = SX.sym("dt")
xS0 = vertcat(n, dn)
ode_p = vertcat(dt, v_c, x_nb)

# # integrator to discretize the system
sys = {}
sys['x'] = xS0
sys['u'] = tau_b
sys['p'] = ode_p
sys['ode'] = ode*dt # Time scaling

intg = integrator('intg', 'rk', sys, 0, 1 , {'simplify':True, 'number_of_finite_elements':50})

res = intg(x0=xS0,u=tau_b, p=ode_p) #evaluate with symbols
x_next = res['xf']

# x_next[6:] = J_@x_next[6:]

In [6]:
x_next.size()

(12, 1)

<h1>Generating discretized state space Linear Model </h1>

In [4]:
states = vertcat(xS0[:6], dn)

A = jacobian(x_next, states)
print(A.size())

B__ = jacobian(x_next, tau_b)
print(B__.size())

(12, 12)
(12, 6)


In [5]:
A_nb_func = Function('A_nb', [dt, x_nb, dn, eul, tau_b, v_c],[A])

current_vel = [0,0,0,0,0,0] # assuming no current flow

A_ = Function('A_', [dt, dn, eul, tau_b],[A_nb_func(dt, J_inv@(dn), dn, eul,tau_b, current_vel)])

A__ = A_(dt, dn, eul, tau_b)

next_state = A__@states + B__@tau_b

x_LTI = Function('LTI', [dt, n, dn, tau_b], [next_state, A__, B__])